In [7]:
import numpy as np
from scipy.optimize import linprog

from solvers import Simplex

In [8]:
class DantzigWolfe:
    """
    Implementa a Decomposição de Dantzig-Wolfe.
    Versão corrigida para a formulação de múltiplos blocos, com uma
    restrição de convexidade por subproblema.
    """

    def __init__(self, master_b, subproblems, tol=1e-7, max_iter=100):
        self.master_b = np.array(master_b)
        self.subproblems = subproblems
        self.num_master_constraints = len(master_b)
        self.num_subproblems = len(self.subproblems)
        self.tol = tol
        self.max_iter = max_iter
        self.big_m = 1e6

        self.proposals = []
        self.rmp_columns = []
        self.rmp_costs = []
        
        self.total_vars = sum(len(sub['c']) for sub in self.subproblems)
        self.sub_var_indices = []
        current_idx = 0
        for sub in self.subproblems:
            num_vars = len(sub['c'])
            self.sub_var_indices.append(slice(current_idx, current_idx + num_vars))
            current_idx += num_vars

        print("Dantzig-Wolfe inicializado.")
        print(f"Número de restrições mestre: {self.num_master_constraints}")
        print(f"Número de subproblemas: {self.num_subproblems}")
        print(f"Número total de variáveis originais: {self.total_vars}")


    def _solve_lp_with_custom_solver(self, c, A_ub=None, b_ub=None, A_eq=None, b_eq=None, bounds=None):
        """
        Wrapper para chamar a classe Simplex customizada e formatar a saída.
        """
        A_list = []
        b_list = []
        signs_list = []

        if A_ub is not None and len(A_ub) > 0:
            A_list.extend(A_ub)
            b_list.extend(b_ub)
            signs_list.extend(['<='] * len(b_ub))
        
        if A_eq is not None and len(A_eq) > 0:
            A_list.extend(A_eq)
            b_list.extend(b_eq)
            signs_list.extend(['='] * len(b_eq))
        
        if not A_list:
             return {'success': False, 'message': 'LP sem restrições.'}

        A = np.array(A_list)
        b = np.array(b_list)
        c_max = -np.array(c)

        solver = Simplex(c_from_parser=c_max, A_from_parser=A, b=b, signs=signs_list)
        result = solver.solve(method='revised')

        if result.get('status') == 'optimal':
            num_decision_vars = len(c_max)
            num_slack_surplus = sum(1 for sign in signs_list if sign != '=')
            full_c_max = np.zeros(num_decision_vars + num_slack_surplus)
            full_c_max[:num_decision_vars] = c_max

            final_basis = result['final_basis_indices']
            B_inv = result['final_B_inv']
            
            c_b_max = full_c_max[final_basis]
            duals_max = c_b_max @ B_inv
            duals = -duals_max

            return {
                'success': True,
                'x': np.array(result['solution'])[:len(c)],
                'fun': -result['value'],
                'duals': duals
            }
        else:
            return {'success': False, 'message': result.get('status', 'Falha no solver.')}


    def _initialize_with_artificials(self):
        print("Inicializando o Problema Mestre com variáveis artificiais...")
        # A base inicial precisa ter m + p colunas
        num_base_cols = self.num_master_constraints + self.num_subproblems
        identity_basis = np.identity(num_base_cols)
        
        for i in range(num_base_cols):
            proposal = {
                'type': 'artificial',
                'cost': self.big_m,
                'column': identity_basis[:, i],
                'x_k_full': np.zeros(self.total_vars) 
            }
            self._add_new_proposal(proposal)


    def _solve_master_problem(self):
        """
        Resolve o Problema Mestre Restrito (PMR) atual.
        Agora com `m` restrições de acoplamento e `p` de convexidade.
        """
        c_rmp = np.array(self.rmp_costs)
        columns_matrix = np.column_stack(self.rmp_columns)
        
        # Separa as restrições de acoplamento (<=) das de convexidade (=)
        A_ub_rmp = columns_matrix[:self.num_master_constraints, :]
        b_ub_rmp = self.master_b
        
        A_eq_rmp = columns_matrix[self.num_master_constraints:, :]
        b_eq_rmp = np.ones(self.num_subproblems)
        
        rmp_result = self._solve_lp_with_custom_solver(
            c=c_rmp,
            A_ub=A_ub_rmp, b_ub=b_ub_rmp,
            A_eq=A_eq_rmp, b_eq=b_eq_rmp
        )
        return rmp_result


    def _solve_pricing_subproblems(self, duals):
        """
        Resolve os subproblemas de pricing usando o dual π correto para cada um.
        """
        lambda_dual = duals[:self.num_master_constraints]
        pi_duals = duals[self.num_master_constraints:] # Agora é um vetor de 'pi'
        
        min_reduced_cost = float('inf')
        best_new_proposal = None
        
        for j, sub in enumerate(self.subproblems):
            c_sub_pricing = sub['c'] - sub['A_master'].T @ lambda_dual
            
            sub_A_ub = sub.get('A_sub')
            sub_b_ub = sub.get('b_sub')
            num_vars = len(sub['c'])
            A_bounds = -np.identity(num_vars)
            b_bounds = np.zeros(num_vars)
            
            sub_A_combined = np.vstack([sub_A_ub, A_bounds])
            sub_b_combined = np.append(sub_b_ub, b_bounds)
            signs = ['<='] * len(sub_b_combined)

            solver = Simplex(c_from_parser=-np.array(c_sub_pricing), A_from_parser=sub_A_combined, b=sub_b_combined, signs=signs)
            sub_result = solver.solve(method='revised')

            if sub_result.get('status') != 'optimal':
                print(f"Aviso: Subproblema {j} não foi resolvido com sucesso.")
                continue
            
            pi_j = pi_duals[j] # Usa o dual de convexidade específico do subproblema
            reduced_cost = -sub_result['value'] - pi_j

            if reduced_cost < min_reduced_cost:
                min_reduced_cost = reduced_cost
                solution_x_local = np.array(sub_result['solution'])[:num_vars]
                
                full_proposal_x = np.zeros(self.total_vars)
                full_proposal_x[self.sub_var_indices[j]] = solution_x_local
                
                # Cria a parte da coluna para as p restrições de convexidade
                convexity_col_part = np.zeros(self.num_subproblems)
                convexity_col_part[j] = 1.0

                best_new_proposal = {
                    'type': 'real',
                    'x_k_full': full_proposal_x,
                    'subproblem_index': j,
                    'cost': sub['c'] @ solution_x_local,
                    'column': np.concatenate([(sub['A_master'] @ solution_x_local), convexity_col_part])
                }
                
        return min_reduced_cost, best_new_proposal


    def _add_new_proposal(self, proposal):
        self.proposals.append(proposal)
        self.rmp_costs.append(proposal['cost'])
        self.rmp_columns.append(proposal['column'])
        if proposal['type'] == 'real':
             print(f"Adicionando nova proposta do subproblema {proposal['subproblem_index']}.")

    def solve(self):
        self._initialize_with_artificials()

        for i in range(self.max_iter):
            print(f"\n--- Iteração {i+1} ---")

            rmp_result = self._solve_master_problem()
            if not rmp_result['success']:
                print(f"ERRO: O Problema Mestre Restrito não pôde ser resolvido. Causa: {rmp_result.get('message')}")
                return None, None

            master_obj_val = rmp_result['fun']
            duals = rmp_result['duals']
            alphas = rmp_result['x']
            
            print(f"Valor da Solução Mestre Atual: {master_obj_val:.6f}")
            print(f"Preços Duais (λ): {duals[:self.num_master_constraints]}")
            print(f"Preços Duais (π): {duals[self.num_master_constraints:]}")
            
            min_reduced_cost, best_new_proposal = self._solve_pricing_subproblems(duals)
            print(f"Custo Reduzido Mínimo encontrado: {min_reduced_cost:.6f}")

            if min_reduced_cost >= -self.tol:
                print("\nCondição de otimalidade atingida. Solução final encontrada.")
                final_solution = self._reconstruct_solution(alphas)
                return final_solution, master_obj_val
            
            self._add_new_proposal(best_new_proposal)

        print("Número máximo de iterações atingido.")
        final_solution = self._reconstruct_solution(alphas)
        return final_solution, master_obj_val


    def _reconstruct_solution(self, alphas):
        final_solution = np.zeros(self.total_vars)
        print("\nReconstruindo a solução final...")
        print("Pesos (α) da solução final:")
        for i, alpha in enumerate(alphas):
            if alpha > self.tol:
                proposal = self.proposals[i]
                print(f"  - Proposta {i} (do subproblema {proposal.get('subproblem_index', 'Artificial')}): α = {alpha:.4f}")
                if proposal['type'] == 'real':
                    final_solution += alpha * proposal['x_k_full']
        return final_solution

In [9]:
# Vetor b da restrição de acoplamento
master_b = [7]

# Lista de subproblemas (neste caso, apenas um)
subproblems_list = [
    {
        'c': np.array([-2, -3]),
        'A_master': np.array([[1, 2]]),
        'A_sub': np.array([[1, 0], [0, 1]]),
        'b_sub': np.array([2, 3]),
        'bounds': [(0, None), (0, None)] # x1 >= 0, x2 >= 0
    }
]

# --- Criando e resolvendo o problema ---
dw_solver = DantzigWolfe(master_b=master_b, subproblems=subproblems_list)
final_solution, final_value = dw_solver.solve()

# --- Exibindo os resultados ---
if final_solution is not None:
    print("\n\n================= RESULTADO FINAL =================")
    print(f"Valor Ótimo da Função Objetivo: {final_value:.6f}")
    print("Valores das Variáveis de Decisão:")
    print(f"  - Solução x: {final_solution}")
    print("===================================================")

Dantzig-Wolfe inicializado.
Número de restrições mestre: 1
Número de subproblemas: 1
Número total de variáveis originais: 2
Inicializando o Problema Mestre com variáveis artificiais...

--- Iteração 1 ---
Valor da Solução Mestre Atual: 1000000.000000
Preços Duais (λ): [-0.]
Preços Duais (π): [1000000.]
Custo Reduzido Mínimo encontrado: -1000013.000000
Adicionando nova proposta do subproblema 0.

--- Iteração 2 ---
Valor da Solução Mestre Atual: 124988.625000
Preços Duais (λ): [-125001.625]
Preços Duais (π): [1000000.]
Custo Reduzido Mínimo encontrado: -1000000.000000
Adicionando nova proposta do subproblema 0.

--- Iteração 3 ---
Valor da Solução Mestre Atual: -11.375000
Preços Duais (λ): [-1.625]
Preços Duais (π): [-0.]
Custo Reduzido Mínimo encontrado: -0.750000
Adicionando nova proposta do subproblema 0.

--- Iteração 4 ---
Valor da Solução Mestre Atual: -11.500000
Preços Duais (λ): [-1.5]
Preços Duais (π): [-1.]
Custo Reduzido Mínimo encontrado: 0.000000

Condição de otimalidade at

In [10]:
# --- Definindo o problema de exemplo com 2 subproblemas ---
master_b = [6, 4]
subproblems_list = [
    # Subproblema 1 (x1, x2)
    {
        'c': np.array([-2, -1]),
        'A_master': np.array([[1, 1], [0, 1]]),
        'A_sub': np.array([[1, 1], [0, 1]]),
        'b_sub': np.array([6, 2]),
    },
    # Subproblema 2 (x3, x4)
    {
        'c': np.array([-3, -1]),
        'A_master': np.array([[1, 1], [2, 1]]),
        'A_sub': np.array([[-1, 1], [1, 1]]),
        'b_sub': np.array([3, 5]),
    }
]

dw_solver = DantzigWolfe(master_b=master_b, subproblems=subproblems_list)
final_solution, final_value = dw_solver.solve()

if final_solution is not None:
    print("\n\n================= RESULTADO FINAL =================")
    print(f"Valor Ótimo da Função Objetivo: {final_value:.6f}")
    print("Valores das Variáveis de Decisão:")
    num_vars_x1 = len(subproblems_list[0]['c'])
    solution_x1 = final_solution[:num_vars_x1]
    solution_x2 = final_solution[num_vars_x1:]
    print(f"  - Solução Bloco 1 (x1, x2): {np.round(solution_x1, 4)}")
    print(f"  - Solução Bloco 2 (x3, x4): {np.round(solution_x2, 4)}")
    print("===================================================")

Dantzig-Wolfe inicializado.
Número de restrições mestre: 2
Número de subproblemas: 2
Número total de variáveis originais: 4
Inicializando o Problema Mestre com variáveis artificiais...

--- Iteração 1 ---
Valor da Solução Mestre Atual: 2000000.000000
Preços Duais (λ): [-0. -0.]
Preços Duais (π): [1000000. 1000000.]
Custo Reduzido Mínimo encontrado: -1000015.000000
Adicionando nova proposta do subproblema 1.

--- Iteração 2 ---
Valor da Solução Mestre Atual: 1599994.000000
Preços Duais (λ): [     -0.  -100001.5]
Preços Duais (π): [1000000. 1000000.]
Custo Reduzido Mínimo encontrado: -1000012.000000
Adicionando nova proposta do subproblema 0.

--- Iteração 3 ---
Valor da Solução Mestre Atual: 933319.333333
Preços Duais (λ): [-166668.66666667  -16667.16666667]
Preços Duais (π): [1000000. 1000000.]
Custo Reduzido Mínimo encontrado: -1000000.000000
Adicionando nova proposta do subproblema 0.

--- Iteração 4 ---
Valor da Solução Mestre Atual: 599986.000000
Preços Duais (λ): [-2.000000e+00 -1

In [11]:
# --- Definindo o problema com um ÚNICO subproblema (blocos unidos) ---
master_b = [6, 4]

subproblems_list = [
    # Subproblema único contendo todas as variáveis e restrições de bloco
    {
        'c': np.array([-2, -1, -3, -1]),
        'A_master': np.array([
            [1, 1, 1, 1],
            [0, 1, 2, 1]
        ]),
        'A_sub': np.array([
            [1, 1,  0,  0],
            [0, 1,  0,  0],
            [0, 0, -1,  1],
            [0, 0,  1,  1]
        ]),
        'b_sub': np.array([6, 2, 3, 5]),
    }
]

dw_solver = DantzigWolfe(master_b=master_b, subproblems=subproblems_list)
final_solution, final_value = dw_solver.solve()

if final_solution is not None:
    print("\n\n================= RESULTADO FINAL =================")
    print(f"Valor Ótimo da Função Objetivo: {final_value:.6f}")
    print("Valores das Variáveis de Decisão:")
    print(f"  - Solução (x1, x2, x3, x4): {np.round(final_solution, 4)}")
    print("===================================================")

Dantzig-Wolfe inicializado.
Número de restrições mestre: 2
Número de subproblemas: 1
Número total de variáveis originais: 4
Inicializando o Problema Mestre com variáveis artificiais...

--- Iteração 1 ---
Valor da Solução Mestre Atual: 1000000.000000
Preços Duais (λ): [-0. -0.]
Preços Duais (π): [1000000.]
Custo Reduzido Mínimo encontrado: -1000027.000000
Adicionando nova proposta do subproblema 0.

--- Iteração 2 ---
Valor da Solução Mestre Atual: 599989.200000
Preços Duais (λ): [     -0.  -100002.7]
Preços Duais (π): [1000000.]
Custo Reduzido Mínimo encontrado: -1000012.000000
Adicionando nova proposta do subproblema 0.

--- Iteração 3 ---
Valor da Solução Mestre Atual: -12.000000
Preços Duais (λ): [-3. -0.]
Preços Duais (π): [6.]
Custo Reduzido Mínimo encontrado: -6.000000
Adicionando nova proposta do subproblema 0.

--- Iteração 4 ---
Valor da Solução Mestre Atual: -14.000000
Preços Duais (λ): [-2.  -0.5]
Preços Duais (π): [-0.]
Custo Reduzido Mínimo encontrado: 0.000000

Condição 